In [1]:
import findspark
findspark.init()

from pyspark.sql.functions import isnan, when, count, col

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

# Configure Spark session

In [5]:
def run():
    from pyspark import SparkContext, SparkConf

    conf = SparkConf()
    conf.setAppName('cars')
    conf.set("spark.driver.memory", "5g")
    conf.set("spark.driver.memoryOverhead", "0.10")
    sc = SparkContext(conf=conf)
    
    return sc

spark = SparkSession(run())

# turn on a dataframe view
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

# Load data

In [9]:
df = spark.read.options(delimiter = '.').csv(path = "used_cars_data.csv", sep = ',',  header = True)

In [10]:
display(df)

vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,engine_type,exterior_color,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,interior_color,isCab,is_certified,is_cpo,is_new,is_oemcpo,latitude,length,listed_date,listing_color,listing_id,longitude,main_picture_url,major_options,make_name,maximum_seating,mileage,model_name,owner_count,power,price,salvage,savings_amount,seller_rating,sp_id,sp_name,theft_title,torque,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
ZACNJABB5KPJ92081,35.1 in,null,null,null,SUV / Crossover,null,Bayamon,null,null,522,00960,[!@@Additional In...,I4,1300.0,I4,Solar Yellow,null,null,True,Jeep,41.2 in,12.7 gal,Gasoline,null,66.5 in,null,177.0,Black,null,null,null,True,null,18.3988,166.6 in,2019-04-06,YELLOW,237132766,-66.1582,https://static.ca...,['Quick Order Pac...,Jeep,5 seats,7.0,Renegade,null,"177 hp @ 5,750 RPM",23141.0,null,0,2.8,370599,Flagship Chrysler,null,"200 lb-ft @ 1,750...",A,9-Speed Automatic...,t83804,Latitude FWD,null,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
SALCJ2FX1LH858117,38.1 in,null,null,null,SUV / Crossover,null,San Juan,null,null,207,00922,[!@@Additional In...,I4,2000.0,I4,Narvik Black,null,null,True,Land Rover,39.1 in,17.7 gal,Gasoline,null,68 in,null,246.0,Black (Ebony),null,null,null,True,null,18.4439,181 in,2020-02-15,BLACK,265946296,-66.0785,https://static.ca...,['Adaptive Cruise...,Land Rover,7 seats,8.0,Discovery Sport,null,"246 hp @ 5,500 RPM",46500.0,null,0,3.0,389227,Land Rover San Juan,null,"269 lb-ft @ 1,400...",A,9-Speed Automatic...,t86759,S AWD,null,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
JF1VA2M67G9829723,35.4 in,null,null,null,Sedan,null,Guaynabo,17.0,null,1233,00969,null,H4,2500.0,H4,None,False,False,True,FIAT,43.3 in,15.9 gal,Gasoline,False,58.1 in,23.0,305.0,None,False,null,null,False,null,18.3467,180.9 in,2017-04-25,UNKNOWN,173473508,-66.1098,null,"['Alloy Wheels', ...",Subaru,5 seats,null,WRX STI,3.0,"305 hp @ 6,000 RPM",46995.0,False,0,null,370467,FIAT de San Juan,False,"290 lb-ft @ 4,000...",M,6-Speed Manual,t58994,Base,null,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
SALRR2RV0L2433391,37.6 in,null,null,null,SUV / Crossover,null,San Juan,null,null,196,00922,[!@@Additional In...,V6,3000.0,V6,Eiger Gray,null,null,True,Land Rover,39 in,23.5 gal,Gasoline,null,73 in,null,340.0,Gray (Ebony/Ebony...,null,null,null,True,null,18.4439,195.1 in,2020-02-26,GRAY,266911050,-66.0785,https://static.ca...,null,Land Rover,7 seats,11.0,Discovery,null,"340 hp @ 6,500 RPM",67430.0,null,0,3.0,389227,Land Rover San Juan,null,"332 lb-ft @ 3,500...",A,8-Speed Automatic...,t86074,V6 HSE AWD,null,AWD,All-Wheel Drive,115 in,87.4 in,2020
SALCJ2FXXLH862327,38.1 in,null,null,null,SUV / Crossover,null,San Juan,null,null,137,00922,[!@@Additional In...,I4,2000.0,I4,Narvik Black,null,null,True,Land Rover,39.1 in,17.7 gal,Gasoline,null,68 in,null,246.0,Black (Ebony),null,null,null,True,null,18.4439,181 in,2020-04-25,BLACK,270957414,-66.0785,https://static.ca...,['Adaptive Cruise...,Land Rover,7 seats,7.0,Discovery Sport,null,"246 hp @ 5,500 RPM",48880.0,null,0,3.0,389227,Land Rover San Juan,null,"269 lb-ft @ 1,400...",A,9-Speed Automatic...,t86759,S AWD,null,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
SALYK2EX1LA261711,37.1 in,null,null,null,SUV / Crossover,null,San Juan,null,null,242,00922,[!@@Additional In...,I4,2000.0,I4,Kaikoura Stone,False,False,True,Land Rover,40.2 in,16.6 gal,Gasoline,False,66.3 in,null,247.0,Brown (Ebony / Eb...,False,null,null,True,null,18.4439,188.9 in,2020-01-11,UNKNOWN,262940541,-66.0785,https://static.ca...,"['Leather Seats',...",Land Rover,5 seats,12.0,Range Rover Velar,null,"247 hp @ 5,500 RPM",66903.0,False,0,3.0,389227,Land Rover San Juan,False,"269 lb-ft @ 1

In [11]:
# save to parquet
df.write.parquet("/tmp/parquet/cars.parquet")

In [12]:
# read from parquet
parDF=spark.read.parquet("/tmp/parquet/cars.parquet")

In [19]:
# shape
print((df.count(), len(df.columns)))

(3000507, 66)


In [14]:
parDF.printSchema()

root
 |-- vin: string (nullable = true)
 |-- back_legroom: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- bed_height: string (nullable = true)
 |-- bed_length: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- city: string (nullable = true)
 |-- city_fuel_economy: string (nullable = true)
 |-- combine_fuel_economy: string (nullable = true)
 |-- daysonmarket: string (nullable = true)
 |-- dealer_zip: string (nullable = true)
 |-- description: string (nullable = true)
 |-- engine_cylinders: string (nullable = true)
 |-- engine_displacement: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- exterior_color: string (nullable = true)
 |-- fleet: string (nullable = true)
 |-- frame_damaged: string (nullable = true)
 |-- franchise_dealer: string (nullable = true)
 |-- franchise_make: string (nullable = true)
 |-- front_legroom: string (nullable = true)
 |-- fuel_tank_volume: string (nullable = tr

In [18]:
parDF.summary().show()


+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------+--------------------+-----------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------

In [16]:
nulls = parDF.select([count(when(isnan(c), c)).alias(c) for c in parDF.columns]).show()
display(nulls)

+---+------------+---+----------+----------+---------+-----+----+-----------------+--------------------+------------+----------+-----------+----------------+-------------------+-----------+--------------+-----+-------------+----------------+--------------+-------------+----------------+---------+-------------+------+--------------------+----------+--------------+-----+------------+------+------+---------+--------+------+-----------+-------------+----------+---------+----------------+-------------+---------+---------------+-------+----------+-----------+-----+-----+-------+--------------+-------------+-----+-------+-----------+------+------------+--------------------+------+---------+-----------------------+------------+--------------------+---------+-----+----+
|vin|back_legroom|bed|bed_height|bed_length|body_type|cabin|city|city_fuel_economy|combine_fuel_economy|daysonmarket|dealer_zip|description|engine_cylinders|engine_displacement|engine_type|exterior_color|fleet|frame_damaged|fran

None

In [14]:
parDF.select([sum(isnan(c)).alias(c) for c in parDF.columns]).show()

TypeError: Column is not iterable